In [46]:

from transformers import BertModel, BertTokenizer
import torch
import pandas as pd
import re

In [26]:
df = pd.read_csv('data_jobads_rn.csv', index_col=None)

In [40]:
df

,title,id,link,date,job_description
0,assistant director of nursing,sj_3c7e64c7996bb9d6,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,Just posted,Silver Stream Healthcare Group offer great emp...
1,clinical nurse manager (cnm),sj_358f1f68cde928c4,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,Hiring ongoing,Create A Better Future For Yourself !!\nRecrui...
2,registered nurse,job_4e16e9830b072344,https://ie.indeed.com/rc/clk?jk=4e16e9830b0723...,Just posted,"Access Healthcare, one of Ireland’s leading He..."
3,staff nurse,job_25a417a6373967b4,https://ie.indeed.com/rc/clk?jk=25a417a6373967...,Just posted,Are you a dedicated and compassionate Staff Nu...
4,clinical research nurse - cardiology (cnm2),job_303eee71cce63f3d,https://ie.indeed.com/rc/clk?jk=303eee71cce63f...,Just posted,Clinical Research Nurse - Cardiology (CNM2)\n\...
...,...,...,...,...,...
559,staff nurse,job_4780b48f19c05dc4,https://ie.indeed.com/rc/clk?jk=4780b48f19c05d...,Posted 30+ days ago,Overview:\nPost Title: Staff Nurse\nPost Statu...
560,assistant director of nursing,job_fb98c2758e8f83d9,https://ie.indeed.com/rc/clk?jk=fb98c2758e8f83...,Posted 30+ days ago,The Grace Healthcare Group are an established ...
561,staff nurse,job_e7284f609fe9a0ec,https://ie.indeed.com/rc/clk?jk=e7284f609fe9a0...,Posted 30+ days ago,"At Grace Healthcare the dignity, comfort and c..."
562,staff nurse,job_3836f4e2da5f81a1,https://ie.indeed.com/rc/clk?jk=3836f4e2da5f81...,Employer\nActive 5 days ago,"A Little Bit About Us\nEstablished in 1917, Le..."


In [49]:
# Function to count words in a given text.
def count_words(text):
    return len(str(text).split())

# Create a new column 'wordCount' based on the 'job_description'.
df['word_count'] = df['job_description'].apply(lambda x: count_words(x))

# Display the DataFrame with the new column
print(df)

                                           title                    id  \
0                  assistant director of nursing   sj_3c7e64c7996bb9d6   
1                   clinical nurse manager (cnm)   sj_358f1f68cde928c4   
2                               registered nurse  job_4e16e9830b072344   
3                                    staff nurse  job_25a417a6373967b4   
4    clinical research nurse - cardiology (cnm2)  job_303eee71cce63f3d   
..                                           ...                   ...   
559                                  staff nurse  job_4780b48f19c05dc4   
560                assistant director of nursing  job_fb98c2758e8f83d9   
561                                  staff nurse  job_e7284f609fe9a0ec   
562                                  staff nurse  job_3836f4e2da5f81a1   
563                         full pin staff nurse   sj_e37186d28df50e47   

                                                  link  \
0    https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl..

In [57]:
filtered_df = df[df['word_count'] > 512]

In [58]:
filtered_df

,title,id,link,date,job_description,new_list,word_count
0,assistant director of nursing,sj_3c7e64c7996bb9d6,https://ie.indeed.com/pagead/clk?mo=r&ad=-6NYl...,Just posted,Silver Stream Healthcare Group offer great emp...,[Silver Stream Healthcare Group offer great em...,563
5,clinical nurse specialist - chest pain,job_4c3a97c49465fb6b,https://ie.indeed.com/rc/clk?jk=4c3a97c49465fb...,Just posted,JOB DESCRIPTION\n\nClinical Nurse Specialist -...,"[JOB DESCRIPTION, , Clinical Nurse Specialist ...",3089
19,NaN,job_c5cbfcd84dd36e5b,https://ie.indeed.com/rc/clk?jk=c5cbfcd84dd36e...,NaN,Are you looking to broaden your nursing skills...,[Are you looking to broaden your nursing skill...,550
20,NaN,job_30234f2c064764c3,https://ie.indeed.com/rc/clk?jk=30234f2c064764...,NaN,"Location: Lucena Clinic, Child & Adolescent Me...","[Location: Lucena Clinic, Child & Adolescent M...",643
21,NaN,job_395a52d19c0403a3,https://ie.indeed.com/rc/clk?jk=395a52d19c0403...,NaN,Role of Subject Matter Expert\nThe Subject Mat...,"[Role of Subject Matter Expert, The Subject Ma...",1148
...,...,...,...,...,...,...,...
550,staff nurse - theatre; anaesthetics and pacu,job_8d6f8c127c0c6ad0,https://ie.indeed.com/rc/clk?jk=8d6f8c127c0c6a...,Posted 30+ days ago,"Job Title; Staff Nurse\nDepartment; Theatre, A...","[Job Title; Staff Nurse, Department; Theatre, ...",859
557,staff nurse - cath lab,job_761d49f6acdcf175,https://ie.indeed.com/rc/clk?jk=761d49f6acdcf1...,Posted 30+ days ago,Job Title: Staff Nurse\nDepartment: Cardiac Ca...,"[Job Title: Staff Nurse, Department: Cardiac C...",997
558,icu staff nurse,job_479ffde2569d7ab3,https://ie.indeed.com/rc/clk?jk=479ffde2569d7a...,Posted 30+ days ago,Overview:\nPost Title: Staff Nurse ICU\n\nLoca...,"[Overview:, Post Title: Staff Nurse ICU, , Loc...",943
560,assistant director of nursing,job_fb98c2758e8f83d9,https://ie.indeed.com/rc/clk?jk=fb98c2758e8f83...,Posted 30+ days ago,The Grace Healthcare Group are an established ...,[The Grace Healthcare Group are an established...,862


In [61]:
for x in range(10, 15):
    print(filtered_df.iat[x, 5])
    print('----------------------------------')

['Grace Healthcare are currently recruiting for a Staff Nurse position for Tara Winthrop Private Clinic in Swords, north Co. Dublin. This is an excellent opportunity for an RGN to work in a progressive environment and further your career within the healthcare industry.', 'All successful candidates must ensure that the highest standards of person centered care is delivered.', 'Shifts are rostered Monday-Sunday and may include night duties.', 'Main Duties and Responsibilities:', 'Utilises assessment skills and techniques to determine patient needs on admission.', 'Completes and documents the nursing admission assessment in an accurate and thorough manner.', 'Assesses patient condition on an ongoing basis.', 'Assesses the need for patient/family instruction.', 'Develops and documents a plan of care based on patient needs', 'Updates the written plan of care based on patient needs.', 'Collaborates with other health team members in planning care.', 'Provides safe, comprehensive nursing care 

In [67]:
a = filtered_df.iat[x, 4]

In [69]:
from summarizer import Summarizer

In [70]:
summarizer = Summarizer()

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

c:\Users\temulenbd\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\temulenbd\.cache\huggingface\hub\models--bert-large-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [71]:
summury = summarizer(a)

c:\Users\temulenbd\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\temulenbd\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [72]:
print(summury)

Job Title: Paediatric Staff Nurse
Department: Paediatrics
Reports to: CNM2 Paediatrics
Date: 2024

Overall Purpose of Job
The purpose of the staff nurse, Paediatric Ambulatory Care is to provide exceptional patient care in an environment where quality, respect, caring and compassion are at the centre of all we do. Key Responsibilities and Deliverables
Excellent Communication and clinical judgement skills to enhance the referral process from General Practitioners (GPs)/Multidisciplinary Team Members to Paediatrics Ambulatory Service. Analyze workloads and skill requirements to meet patient/family care needs. Deputize for the Clinical Coordinator in their absence
Serves as a resource person for other staff, family and patients attending the Service. Assists with the assessment, planning, implementation and evaluation of patient care as required. Performs techniques and routines in the specialty service area. Documents nursing care in all assigned patient records. Reassesses and documents

In [5]:
model = BertModel.from_pretrained('bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

c:\Users\temulenbd\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\temulenbd\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [20]:
text_1st = df.iat[0, 4]

In [21]:
text_1st

'Silver Stream Healthcare Group offer great employment and training opportunities. We are committed to attracting and retaining professional and dedicated staff to help us provide the highest level of care possible to our residents and a peace of mind to their families.\nOur Assistant Director of Nursing will carry out a range of duties to ensure that the care and well-being of residents is met satisfactorily, including assisting in future planning, supervision and control of all staff and deputising for Director of Nursing in their absence at our Leeson Park Nursing Home (10 Leeson Park, Dublin 6, D06 TC65).\nPrincipal Responsibilities:\nSupervising, and monitoring all staff in the course of their duties. Complete staff weekly allocation sheets.\nSupervising and controlling arrangements for resident’s admissions/discharges. Carry out home viewings as required. Welcome new residents to the home. Meet with families as required.\nResponsible for ensuring correct infection control procedu

In [22]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [23]:
tokens = tokenizer.tokenize(text_1st)

In [25]:
len(tokens)

728